In [1]:
import foldermerge
from pathlib import Path

root = Path().absolute().parent / "tests"
foldermerge.core.RESULTS_PATH = root / "results"
foldermerge.clear_results()

In [2]:
fm = foldermerge.FolderMerger(root / "integration" / "fixtures" / "mainfolder", root / "integration" / "fixtures" / "dupefolder1")

loading <FolderChecker 626ed55d at c:\Users\Timothe\Documents\python_scripts\foldermerge\tests\integration\fixtures\mainfolder>
loading <FolderChecker fbd1a246 at c:\Users\Timothe\Documents\python_scripts\foldermerge\tests\integration\fixtures\dupefolder1>
Finding all files in the repo c:\Users\Timothe\Documents\python_scripts\foldermerge\tests\integration\fixtures\mainfolder


Searching: 2it [00:00, ?it/s]


Claculating hashes for 7 files :


  0%|          | 0/7 [00:00<?, ?it/s]

100%|██████████| 7/7 [00:00<00:00, 39.76it/s]

...

saving <FolderChecker fbd1a246 at c:\Users\Timothe\Documents\python_scripts\foldermerge\tests\integration\fixtures\dupefolder1>
loading <FolderComparator fbd1a246_vs_626ed55d>
Comparing names:


100%|██████████| 9/9 [00:00<00:00, 2192.40it/s]


Comparing hashes:


100%|██████████| 9/9 [00:00<00:00, 5574.24it/s]

saving <FolderComparator fbd1a246_vs_626ed55d>


In [3]:
print(fm.report()[0])

Report of contents for repo fbd1a246 at c:\Users\Timothe\Documents\python_scripts\foldermerge\tests\integration\fixtures\dupefolder1:
 - 9 total files found.
 - 4 identical files (will be deleted)
 - 1 inexistant files (will be copied in main)
 - 3 moved files (same content, different location) (tbd)
 - 1 changed files (same location, different content) (tbd)



In [4]:
comp = fm.folders.child(0).comparisons[fm.folders.main.name]

In [6]:
comp.get_changed_files()

,filename,name,ext,fullpath,relpath,dirs,ctime,mtime,time,hash,name_matches,content_matches
uuid,,,,,,,,,,,,
2901106866045615176,content_changed_file_1.jpg,content_changed_file_1,.jpg,c:\Users\Timothe\Documents\python_scripts\fold...,images\content_changed_file_1.jpg,[images],1.706356e+09,1.707252e+09,1.707252e+09,00add8e211807e550a2a64807720543ac99813e9,[-6383645297580544904],[]


In [7]:
df = comp.get_identical_files()
df

,filename,name,ext,fullpath,relpath,dirs,ctime,mtime,time,hash,name_matches,content_matches
uuid,,,,,,,,,,,,
8526814694241885902,conflict_copy_file_1.jpg,conflict_copy_file_1,.jpg,c:\Users\Timothe\Documents\python_scripts\fold...,images\conflict_copy_file_1.jpg,[images],1.706356e+09,1.706356e+09,1.706356e+09,0efc03bf785344023dd734b7119a91182178b8f0,[6384961973676772252],[6384961973676772252]
4674087730968615740,identical_file_1.jpg,identical_file_1,.jpg,c:\Users\Timothe\Documents\python_scripts\fold...,images\identical_file_1.jpg,[images],1.706356e+09,1.706356e+09,1.706356e+09,b3b29122e5dda4203d1e6fadc8ad34eb56e69845,[-7028881286459164115],[-7028881286459164115]
-2359237790709598665,identical_file_2.jpg,identical_file_2,.jpg,c:\Users\Timothe\Documents\python_scripts\fold...,images\identical_file_2.jpg,[images],1.706356e+09,1.706356e+09,1.706356e+09,c939af2ee954153e0290f57ae7761812754f7caf,[-7542450700712088608],[-7542450700712088608]
7484861899368366695,identical_file_3.jpg,identical_file_3,.jpg,c:\Users\Timothe\Documents\python_scripts\fold...,images\identical_file_3.jpg,[images],1.706356e+09,1.706356e+09,1.706356e+09,7dbd73a7133ccbd552f776da91da227ff5e121d0,[-406773629693236663],[-406773629693236663]


In [ ]:
def check_date(row, reference_data, ):
    matching_file_index = row

## Algorithm :

| content match | name match | date case      | case name         | action to take               |
| :-----------: | :--------: | :------------: | :---------------: | :--------------------------: |
| True          | True       | more recent    | identical         | *ignore* (keep ref)            |
| True          | True       | more old       | identical         | *ignore* (keep ref)            |
| True          | False      | more recent    | renamed or moved  | **copy** to ref (if context same)|
| True          | False      | more old       | renamed or moved  | *ignore* (keep ref)            |
| False         | True       | more recent    | updated modified  | **copy** to ref (overwrite)      |
| False         | True       | more old       | outdated modified | *ignore* (keep ref)            |
| False         | False      | -              | -                 | **copy** to ref (no file there)  |

Details for renamed or moved :

| date case   | context details | case name | action to take |
| :---------: | :-------------: | :-------: | :------------: |
| more recent | tbd             |           |                |
| more old    | tbd             |           |                |

In [10]:
index_ref = df.iloc[0].name_matches[0]
main_data = fm.folders.main.data
main_data.loc[index_ref]

filename                             conflict_copy_file_1.jpg
name                                     conflict_copy_file_1
ext                                                      .jpg
fullpath    c:\Users\Timothe\Documents\python_scripts\fold...
relpath                       images\conflict_copy_file_1.jpg
dirs                                                 [images]
ctime                                       1706355702.789143
mtime                                       1706355702.789143
time                                        1706355702.789143
hash                 0efc03bf785344023dd734b7119a91182178b8f0
Name: 6384961973676772252, dtype: object

In [11]:
main_data

,filename,name,ext,fullpath,relpath,dirs,ctime,mtime,time,hash
uuid,,,,,,,,,,
6384961973676772252,conflict_copy_file_1.jpg,conflict_copy_file_1,.jpg,c:\Users\Timothe\Documents\python_scripts\fold...,images\conflict_copy_file_1.jpg,[images],1.706356e+09,1.706356e+09,1.706356e+09,0efc03bf785344023dd734b7119a91182178b8f0
-6383645297580544904,content_changed_file_1.jpg,content_changed_file_1,.jpg,c:\Users\Timothe\Documents\python_scripts\fold...,images\content_changed_file_1.jpg,[images],1.706356e+09,1.706356e+09,1.706356e+09,cf94148b62d3b89e6f7c1159e19383874304efd4
-7028881286459164115,identical_file_1.jpg,identical_file_1,.jpg,c:\Users\Timothe\Documents\python_scripts\fold...,images\identical_file_1.jpg,[images],1.706356e+09,1.706356e+09,1.706356e+09,b3b29122e5dda4203d1e6fadc8ad34eb56e69845
-7542450700712088608,identical_file_2.jpg,identical_file_2,.jpg,c:\Users\Timothe\Documents\python_scripts\fold...,images\identical_file_2.jpg,[images],1.706356e+09,1.706356e+09,1.706356e+09,c939af2ee954153e0290f57ae7761812754f7caf
-406773629693236663,identical_file_3.jpg,identical_file_3,.jpg,c:\Users\Timothe\Documents\python_scripts\fold...,images\identical_file_3.jpg,[images],1.706356e+09,1.706356e+09,1.706356e+09,7dbd73a7133ccbd552f776da91da227ff5e121d0
-6051083323529826513,unchanged_name_file_1.jpg,unchanged_name_file_1,.jpg,c:\Users\Timothe\Documents\python_scripts\fold...,images\unchanged_name_file_1.jpg,[images],1.706356e+09,1.706356e+09,1.706356e+09,58b24d71528352e2b4b46f497f3b0d87e5775877
-7524050800546347642,unchanged_name_file_2.jpg,unchanged_name_file_2,.jpg,c:\Users\Timothe\Documents\python_scripts\fold...,images\unchanged_name_file_2.jpg,[images],1.706356e+09,1.706356e+09,1.706356e+09,5877ffde9d973af3a6c17254c66d54602cd634e9


In [11]:
comp.get_inexistant_files()

,filename,name,ext,fullpath,relpath,dirs,ctime,mtime,time,hash,name_matches,content_matches
uuid,,,,,,,,,,,,
-8378644071822432228,inexistant_file_1.jpg,inexistant_file_1,.jpg,c:\Users\Timothe\Documents\python_scripts\fold...,images\inexistant_file_1.jpg,[images],1.706356e+09,1.706356e+09,1.706356e+09,361151bd95808a23fdcff4d4adeb55e186998d48,[],[]


In [12]:
comp.get_moved_files()

,filename,name,ext,fullpath,relpath,dirs,ctime,mtime,time,hash,name_matches,content_matches
uuid,,,,,,,,,,,,
3568062500631960647,conflict_copy_duplicated_file_1.jpg,conflict_copy_duplicated_file_1,.jpg,c:\Users\Timothe\Documents\python_scripts\fold...,images\conflict_copy_duplicated_file_1.jpg,[images],1.706356e+09,1.706356e+09,1.706356e+09,0efc03bf785344023dd734b7119a91182178b8f0,[],[6214847964681687045]
-1609281068694028568,renamed_file_1.jpg,renamed_file_1,.jpg,c:\Users\Timothe\Documents\python_scripts\fold...,images\renamed_file_1.jpg,[images],1.706356e+09,1.706356e+09,1.706356e+09,58b24d71528352e2b4b46f497f3b0d87e5775877,[],[-275924656751009016]
7304253802138092403,renamed_file_2.jpg,renamed_file_2,.jpg,c:\Users\Timothe\Documents\python_scripts\fold...,images\renamed_file_2.jpg,[images],1.706356e+09,1.706356e+09,1.706356e+09,5877ffde9d973af3a6c17254c66d54602cd634e9,[],[8995852696255613759]
